In [1]:
!pip install pygame
!pip install pygame_widgets
!pip install svgpathtools

In [2]:
import pygame
import pygame_widgets
from pygame_widgets.slider import Slider
from pygame_widgets.textbox import TextBox
from svgpathtools import svg2paths
from numpy import cos, sin, pi, sqrt, arctan2
import pygame.draw as draw
from pygame import gfxdraw
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.10.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def fourier_term(signals, k):
  N = len(signals)
  X_k = complex()
  for n in range(N):
    theta = 2 * pi * k * n / N
    X_k += complex(cos(theta), -sin(theta)) * signals[n]
  return X_k / N

def fourier_transform(signals):  
  return [fourier_term(signals, k) for k in range(len(signals))]

In [4]:
class Wave:
  def __init__(self, frequency, amplitude, phase):
    self.frequency = frequency
    self.amplitude = amplitude
    self.phase = phase
  def __repr__(self):
    return f"Wave(freq={self.frequency}, amp={self.amplitude}, phase={self.phase})"

  @staticmethod
  def from_fourier_term(k, cmplx_num):
    r = cmplx_num.real
    i = cmplx_num.imag
    return Wave(k, sqrt(r * r + i * i), arctan2(i, r))

def generate_wave_info(signals):
  fourier_terms = fourier_transform(signals)
  return [Wave.from_fourier_term(k, term) for k, term in enumerate(fourier_terms)]

In [5]:
def read_file(filename):
  file = open(filename, "r")
  return [[float(num) for num in line.split(", ")] for line in file]

In [6]:
drawing = read_file('assets/capitalE.txt')
xs = [point[0] * 2 for point in drawing]
ys = [point[1] * 2 for point in drawing]

In [7]:
# Load SVG file
paths, attributes = svg2paths('assets/R.svg')
xsR = []
ysR = []

# Extracting coordinates from paths
for path in paths:
    for segment in path:
        # Extract coordinates from the segment
        for point in segment:
            x, y = point.real, point.imag
            xsR.append(x)
            ysR.append(y)

In [8]:
paths, attributes = svg2paths('assets/A.svg')
xsA = []
ysA = []

# Extracting coordinates from paths
for path in paths:
    for segment in path:
        # Extract coordinates from the segment
        for point in segment:
            x, y = point.real, point.imag
            xsA.append(x)
            ysA.append(y)

### Importing data for each letter and formatting into dictionary

Function to extract data from svg file and format into x and y position arrays

In [9]:
def extract(paths):
    xs = []
    ys = []
    for path in paths:
        for segment in path:
            # Extract coordinates from the segment
            for point in segment:
                x, y = point.real, point.imag
                xs.append(x)
                ys.append(y)
    return xs,ys

In [10]:
max_y = 0
dc = dict()
for i in range(26):
    lowercase_letter = chr(97+i)
    uppercase_letter = chr(65+i)
    lower_route = f"assets/letters/lower/{lowercase_letter}.svg"
    paths,attributes = svg2paths(lower_route)
    xs, ys = extract(paths)
    max_y = max(max_y, max(ys))
    dc[lowercase_letter] = (xs, ys)
    upper_route = f"assets/letters/upper/{uppercase_letter}.svg"
    paths,attributes = svg2paths(upper_route)
    xs, ys = extract(paths)
    max_y = max(max_y, max(ys))
    dc[uppercase_letter] = (xs, ys)

dc[' '] = [0, 30], [max_y, max_y]
letter_height = max_y

### Reading in user input

In [11]:

text = input('What would you like to be written?\n')

points_x = []
points_y = []

offset = 0
for i in text:
    points_x.append(offset)
    points_y.append(letter_height)
    points_x += [x + offset for x in dc[i][0]]
    curr_max_y = max(dc[i][1])
    points_y += [letter_height - curr_max_y + y for y in dc[i][1]]
    points_x.append(points_x[-1])
    points_y.append(letter_height)
    offset = max(points_x)

points_x = [point - offset / 2 for point in points_x]
points_y = [point - letter_height / 2 for point in points_y]
x_waves = sorted(generate_wave_info(points_x), key=lambda x: -x.amplitude)
y_waves = sorted(generate_wave_info(points_y), key=lambda x: -x.amplitude)

In [12]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  
  def __add__(self, other):
    return Point(self.x + other.x, self.y + other.y)
  
  def __sub__(self, other):
    return Point(self.x - other.x, self.y - other.y)
  
  def __iadd__(self, other):
    self.x += other.x
    self.y += other.y
    return self

  def __truediv__(self, num):
    return Point(self.x / num, self.y / num)
  
  def __mul__(self, num):
    return Point(self.x * num, self.y * num)
  
  def __repr__(self):
    return f"({self.x}, {self.y})"
  
  def as_tuple(self):
    return (self.x, self.y)

In [13]:
# pygame setup
pygame.init()

background_color = (255, 255, 255)
circle_color = (100, 100, 100)
radius_color = (200, 200, 200)
pointer_line_color = (255, 100, 100)

old_line_color = (100, 100, 100)
drawing_color = (0, 0, 50)
widget_color = (0, 122, 255)

screensize = Point(1280, 720)
screencenter = screensize / 2
screen = pygame.display.set_mode(screensize.as_tuple())
clock = pygame.time.Clock()
running = True
time = 0
points = []
oldPoints = []
epsilon = pi / 8

slider_x = screencenter.x - 50

slider = Slider(screen, slider_x, 600, 100, 10, min=1, max=5, step=1, initial=1, handleColour=widget_color)
output = TextBox(screen, slider_x, 630, 0, 0, fontSize=12)
speed_label = TextBox(screen, slider_x, 642, 0, 0, fontSize=12)
speed_label.setText('(Higher Speed = Lower Resolution)')
speed_label.disable()
output.disable()

while running:

    output.setText(f'Speed: {slider.getValue()}x')

    events = pygame.event.get()
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in events:
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
    # fill the screen with a color to wipe away anything from last frame
    screen.fill(background_color)

    currPointX = Point(screencenter.x, 100)
    for wave in x_waves:
        prevPoint = currPointX

        freq = wave.frequency
        radius = wave.amplitude
        phase = wave.phase

        offsetPoint = Point(cos(freq * time + phase), sin(freq * time + phase)) * radius
        currPointX = prevPoint + offsetPoint
        draw.aaline(screen, radius_color, prevPoint.as_tuple(), currPointX.as_tuple(), 2)
        radiusPoint = Point(radius, radius)
        topLeft = prevPoint - radiusPoint
        gfxdraw.aacircle(screen, int(prevPoint.x), int(prevPoint.y), int(radius), circle_color)
    
    currPointY = Point(100, screencenter.y)
    for wave in y_waves:
        prevPoint = currPointY

        freq = wave.frequency
        radius = wave.amplitude
        phase = wave.phase

        offsetPoint = Point(cos(freq * time + phase + (pi / 2)), sin(freq * time + phase + (pi / 2))) * radius
        currPointY = prevPoint + offsetPoint
        draw.aaline(screen, radius_color, prevPoint.as_tuple(), currPointY.as_tuple(), 2)
        radiusPoint = Point(radius, radius)
        topLeft = prevPoint - radiusPoint
        gfxdraw.aacircle(screen, int(prevPoint.x), int(prevPoint.y), int(radius), circle_color)

    time += slider.getValue() * (2 * pi / len(x_waves))
        
    finalPoint = (currPointX.x, currPointY.y)
    draw.aaline(screen, pointer_line_color, currPointX.as_tuple(), finalPoint)
    draw.aaline(screen, pointer_line_color, currPointY.as_tuple(), finalPoint)
    
    points.append((currPointX.x, currPointY.y))
    if (time >= 2 * pi - epsilon):
        oldPoints.append(points[0])
        points = points[1:]
    if len(oldPoints) >= 250:
        oldPoints = oldPoints[1:]

    if len(oldPoints) > 1:
        draw.aalines(screen, old_line_color, False, oldPoints)
    if len(points) > 1:
        draw.aalines(screen, drawing_color, False, points)
    
    pygame_widgets.update(events)
    pygame.display.flip()

    clock.tick(60)  # limits FPS to 60

pygame.quit()